# 示例1: YoutubeDNN-ml_1m
https://github.com/shenweichen/DeepMatch/blob/master/examples/colab_MovieLen1M_YoutubeDNN.ipynb
    
下面已大家比较熟悉的YoutubeDNN为例子，给大家介绍如何使用deepmatch进行召回模型的训练，用户和物品向量的导出，以及使用faiss进行近似最近邻搜索。

整段代码不到100行，可以是非常的方便进行学习和使用了～

__运行环境 tf = 1.14.0, tf2会报错!!!__

## 任务
通过用户的历史点击，来预测用户下一次点击的item_id

X = [(user_id, hist_items_list, next_item, items_list_length),]

Y = [1, 0...]  二分类

In [1]:
import tensorflow as tf
print(tf.__version__)

/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/luoyonggui/anaconda3/envs/tf14

1.14.0


/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/luoyonggu

## 导入需要的库

In [2]:
import pandas as pd
from deepctr.inputs import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss


## preprocess

In [3]:
import random
import numpy as np
from tqdm import tqdm
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

### gen_data_set

In [4]:
def gen_data_set(data, negsample: int=0):
    """
    negsample: 负样本数相对于正样本数的倍数
    """
    data.sort_values("timestamp", inplace=True)
    item_ids = data['movie_id'].unique()

    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['movie_id'].tolist()  # 用户看过的电影列表
        rating_list = hist['rating'].tolist()  # 用户看过的电影评分列表

        if negsample > 0:
            # 产生负样本，策略：从用户没有评分的items中随机有放回采样
            candidate_set = list(set(item_ids) - set(pos_list))
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            if i != len(pos_list) - 1:
                # (user_id, items_list, next_item, label, items_list_length, next_item_rating)
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1,len(hist[::-1]),rating_list[i]))
                for negi in range(negsample):
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1])))
            else:
                test_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1]),rating_list[i]))

    random.shuffle(train_set)
    random.shuffle(test_set)

    print(len(train_set[0]),len(test_set[0]))

    return train_set,test_set

### gen_model_input

In [5]:
def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"user_id": train_uid, "movie_id": train_iid, "hist_movie_id": train_seq_pad,
                         "hist_len": train_hist_len}

    for key in ["gender", "age", "occupation", "zip"]:
        train_model_input[key] = user_profile.loc[train_model_input['user_id']][key].values

    return train_model_input,train_label

## 读取数据

In [6]:
data_path = "../../data/rs/"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml_1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml_1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml_1m/movies.dat',sep='::',header=None,names=mnames)

data = pd.merge(pd.merge(ratings,movies),user)#.iloc[:10000]


/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warnin

In [7]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


## 构建特征列，训练模型，导出embedding

In [8]:
#data = pd.read_csvdata = pd.read_csv("./movielens_sample.txt")
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", ]
SEQ_LEN = 50
negsample = 0

# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip']
feature_max_idx = {}
for feature in features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1

user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')

item_profile = data[["movie_id"]].drop_duplicates('movie_id')

user_profile.set_index("user_id", inplace=True)

user_item_list = data.groupby("user_id")['movie_id'].apply(list)

In [9]:
user_item_list.head(2)

user_id
1    [1105, 640, 854, 3178, 2163, 1108, 1196, 2600,...
2    [1105, 2890, 2129, 1783, 1118, 1849, 1155, 126...
Name: movie_id, dtype: object

In [10]:
data.head(2)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1105,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1,1,11,1589
1,1,640,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,1,1,11,1589


In [11]:
train_set, test_set = gen_data_set(data, negsample)

100%|██████████| 6040/6040 [00:14<00:00, 416.82it/s]


6 6


In [12]:
train_set[:1]

[(1010,
  [1710,
   2199,
   2797,
   3652,
   2073,
   2972,
   3092,
   3703,
   3665,
   2410,
   3199,
   1598,
   498,
   3522,
   2213,
   2598,
   3398,
   1273,
   3221,
   2175,
   1502,
   2217,
   3384,
   2155,
   88,
   3592,
   3461,
   3154,
   2439,
   2333,
   2334,
   3044,
   2744,
   1112,
   2740,
   596,
   3462,
   2335,
   1289,
   2789,
   1775,
   1213,
   2174,
   1132,
   873,
   1464,
   2933,
   3706,
   3504,
   3378,
   3587,
   2756,
   2137,
   2945,
   3249,
   2702,
   3513,
   3281,
   3551,
   3509,
   2799,
   1774,
   3190,
   3134,
   1009,
   1151,
   2600,
   3494,
   1144,
   850,
   1125,
   1108,
   863,
   1119,
   852,
   853,
   3430,
   2737,
   848,
   1172,
   1114,
   2717,
   1105,
   714,
   3236,
   1501,
   1161,
   1185,
   1176,
   1296,
   803,
   1060,
   1171,
   528,
   1283,
   3012,
   1942,
   3469,
   1941,
   2219,
   2266,
   1797,
   1791,
   2441,
   2216,
   2276,
   2656,
   2074,
   2537,
   2210,
   2229,
   251

In [13]:
train_seq = [line[1] for line in train_set]

In [14]:
train_seq[:2]

[[1710,
  2199,
  2797,
  3652,
  2073,
  2972,
  3092,
  3703,
  3665,
  2410,
  3199,
  1598,
  498,
  3522,
  2213,
  2598,
  3398,
  1273,
  3221,
  2175,
  1502,
  2217,
  3384,
  2155,
  88,
  3592,
  3461,
  3154,
  2439,
  2333,
  2334,
  3044,
  2744,
  1112,
  2740,
  596,
  3462,
  2335,
  1289,
  2789,
  1775,
  1213,
  2174,
  1132,
  873,
  1464,
  2933,
  3706,
  3504,
  3378,
  3587,
  2756,
  2137,
  2945,
  3249,
  2702,
  3513,
  3281,
  3551,
  3509,
  2799,
  1774,
  3190,
  3134,
  1009,
  1151,
  2600,
  3494,
  1144,
  850,
  1125,
  1108,
  863,
  1119,
  852,
  853,
  3430,
  2737,
  848,
  1172,
  1114,
  2717,
  1105,
  714,
  3236,
  1501,
  1161,
  1185,
  1176,
  1296,
  803,
  1060,
  1171,
  528,
  1283,
  3012,
  1942,
  3469,
  1941,
  2219,
  2266,
  1797,
  1791,
  2441,
  2216,
  2276,
  2656,
  2074,
  2537,
  2210,
  2229,
  2513,
  3157,
  2075,
  3158,
  1275,
  2590,
  3467,
  2279,
  2735,
  2531,
  3409,
  3164,
  2467,
  2259,
  2694,
  187

In [15]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [16]:
pad_sequences(train_seq[:2], maxlen=50, padding='post', truncating='post', value=0)

array([[1710, 2199, 2797, 3652, 2073, 2972, 3092, 3703, 3665, 2410, 3199,
        1598,  498, 3522, 2213, 2598, 3398, 1273, 3221, 2175, 1502, 2217,
        3384, 2155,   88, 3592, 3461, 3154, 2439, 2333, 2334, 3044, 2744,
        1112, 2740,  596, 3462, 2335, 1289, 2789, 1775, 1213, 2174, 1132,
         873, 1464, 2933, 3706, 3504, 3378],
       [ 952, 2224,  574, 3040, 2491, 3292, 1832, 1304, 1012,  800,  487,
        1944, 2058, 1822, 1282, 2456, 3274, 2073, 1450, 1527, 2324, 1900,
        3032, 3168, 3083, 2192, 3319,  964, 3167, 2591, 1049, 1898, 3220,
        1505, 1821, 2231, 3219,  584, 1789, 2846, 2160, 1131, 3166, 2129,
        2739, 2708, 2981,  102, 2163,    1]], dtype=int32)

In [17]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

In [18]:
train_model_input

{'user_id': array([1010,  749, 3622, ..., 1889, 1613, 4869]),
 'movie_id': array([ 223, 2213, 1538, ...,  537,  965, 1203]),
 'hist_movie_id': array([[1710, 2199, 2797, ..., 3706, 3504, 3378],
        [ 952, 2224,  574, ...,  102, 2163,    1],
        [  39,  285,  873, ..., 1008, 1100, 2547],
        ...,
        [ 160,  165,  936, ..., 2506, 1177, 3375],
        [2206,  893, 1839, ..., 2740, 1013, 1112],
        [1289, 1060, 1125, ..., 1935, 2177, 1204]], dtype=int32),
 'hist_len': array([948, 224, 113, ..., 501, 787, 227]),
 'gender': array([2, 2, 2, ..., 2, 2, 1]),
 'age': array([3, 4, 2, ..., 2, 2, 5]),
 'occupation': array([ 1, 19,  2, ..., 15, 21,  7]),
 'zip': array([ 431, 1962,  135, ..., 2526, 2112,  244])}

In [19]:
SparseFeat('user_id', feature_max_idx['user_id'], 16)

SparseFeat(name='user_id', vocabulary_size=6041, embedding_dim=16, use_hash=False, dtype='int32', embedding_name='user_id', group_name='default_group')

In [20]:
# 2.count #unique features for each sparse field and generate feature config for sequence feature

embedding_dim = 32

user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("gender", feature_max_idx['gender'], 16),
                        SparseFeat("age", feature_max_idx['age'], 16),
                        SparseFeat("occupation", feature_max_idx['occupation'], 16),
                        SparseFeat("zip", feature_max_idx['zip'], 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]

## Define Model and train
召回模型的网络结构

In [21]:
# 3.Define Model and train

K.set_learning_phase(True)

model = YoutubeDNN(user_feature_columns, item_feature_columns, num_sampled=200, user_dnn_hidden_units=(128,64, embedding_dim))
# model = MIND(user_feature_columns,item_feature_columns,dynamic_k=True,p=1,k_max=2,num_sampled=5,user_dnn_hidden_units=(64,16),init_std=0.001)

model.compile(optimizer="adam", loss=sampledsoftmaxloss)  # "binary_crossentropy")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_index (EmbeddingIndex (3707,)              0           movie_id[0][0]                   
__________________________________________________________________________________________________
hist_movie_id (InputLayer)      [(None, 50)]         0                                            
__________________________________________________________________________________________________
sparse_seq_emb_hist_movie_id (E multiple             118624      hist_movie_id[0][0]              
                                                                 embedding_index[0][0]        

In [23]:
import tensorflow as tf

In [31]:
tf.keras.utils.plot_model(model)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [45]:
history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=24, verbose=1, validation_split=0.0, )

Epoch 1/24
988129/988129 [==============================] - 22s 22us/sample - loss: 3.4355
Epoch 2/24
988129/988129 [==============================] - 21s 21us/sample - loss: 3.2733
Epoch 3/24
988129/988129 [==============================] - 20s 20us/sample - loss: 3.1565
Epoch 4/24
988129/988129 [==============================] - 22s 22us/sample - loss: 3.0615
Epoch 5/24
988129/988129 [==============================] - 21s 22us/sample - loss: 2.9840
Epoch 6/24
988129/988129 [==============================] - 18s 19us/sample - loss: 2.9255
Epoch 7/24
988129/988129 [==============================] - 18s 19us/sample - loss: 2.8788
Epoch 8/24
988129/988129 [==============================] - 23s 23us/sample - loss: 2.8405
Epoch 9/24
988129/988129 [==============================] - 19s 19us/sample - loss: 2.8063
Epoch 10/24
988129/988129 [==============================] - 19s 19us/sample - loss: 2.7827
Epoch 11/24
988129/988129 [==============================] - 20s 20us/sample - loss: 2.75

训练完整后，由于在实际使用时，我们需要根据当前的用户特征实时产生用户侧向量，并对物品侧向量构建索引进行近似最近邻查找。这里由于是离线模拟，所以我们导出所有待测试用户的表示向量，和所有物品的表示向量。


## Generate user features for testing and full item features for retrieval

In [46]:
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

# 以下两行是deepmatch中的通用使用方法，分别获得用户向量模型和物品向量模型
user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

# 输入对应的数据拿到对应的向量
user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(6040, 32)
(3706, 32)


## 使用faiss进行ANN查找并评估结果
[可选的]如果有安装faiss库的同学，可以体验以下将上一步导出的物品向量构建索引，然后用用户向量来进行ANN(Approximate Nearest Neighbor，近似最近邻)查找并评估效果

In [47]:
# ! pip install faiss-cpu
!pip freeze | grep faiss

faiss-cpu==1.6.3


In [48]:
test_true_label = {line[0]:[line[2]] for line in test_set}  # user_id: [next_item_id]

import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(user_embs, 50)
s = []
hit = 0
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['movie_id'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))

## result
训练了24个epoch，top50召回达到了0.29，